<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---- Imports ----
import time
import difflib
from datetime import datetime
import re
import google.generativeai as genai
from google.colab import userdata

# Load other project modules using importnb
from importnb import Notebook
with Notebook():
    import mqqt_sim_indoor as indoor
    import mqqt_sim_outdoor as outdoor
    import user_controller as uc
    import FireBase as fb

# Gemini API key (ensure this is set in your Colab secrets)
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")

In [ ]:
"""
Sensor Meta Table
-----------------
Defines metadata for all sensors in the OptiLine system.
This information is used to answer questions about sensor functions, valid value ranges, and sensor descriptions.
"""

SENSORS_META = {
    "temperature": {
        "label": "Temperature",
        "units": "°C",
        "locations": ["indoor", "outdoor"],
        "description": "Measures ambient air temperature.",
        "normal_range": [18, 26],
        "details": "Temperature sensors are used to monitor the climate in the production environment and ensure optimal conditions."
    },
    "humidity": {
        "label": "Humidity",
        "units": "%",
        "locations": ["indoor", "outdoor"],
        "description": "Measures relative air humidity.",
        "normal_range": [30, 60],
        "details": "Humidity sensors help prevent issues such as condensation, corrosion, or material degradation."
    },
    "pressure": {
        "label": "Pressure",
        "units": "hPa",
        "locations": ["indoor"],
        "description": "Measures atmospheric pressure.",
        "normal_range": [1000, 1025],
        "details": "Pressure sensors provide data that can be used for weather analysis or to ensure safety standards in the facility."
    },
    "dlight": {
        "label": "Dlight",
        "units": "Lux",
        "locations": ["outdoor"],
        "description": "Measures light intensity (illuminance).",
        "normal_range": [10000, 50000],
        "details": "Dlight sensors can be used to monitor daylight exposure, useful for environmental analysis."
    },
    "distance": {
        "label": "Distance",
        "units": "mm",
        "locations": ["indoor"],
        "description": "Ultrasonic sensor detects distance changes (e.g., movement nearby).",
        "normal_range": [0, 500],
        "details": "Distance sensors are used for safety, automation, and detecting human or object presence."
    }
}


In [ ]:
"""
User Guide, FAQ, and System Context
-----------------------------------
This context is included in every prompt to Gemini, providing the model with up-to-date knowledge about the OptiLine Dashboard,
the CIM & Robotics Laboratory, available sensors, and operational policies.
"""

OPTI_BOT_CONTEXT = """
Welcome to the OptiLine Dashboard – a real-time cloud-based interface for engineers in the CIM & Robotics Lab.

Main Features:
• Live Sensors Dashboard – Monitor real-time data from indoor and outdoor sensors.
• Statistics Panel – Analyze historical sensor data with interactive plots.
• MQTT Search Engine – Search technical documentation from MQTT.org.
• Fault Simulator – Practice troubleshooting simulated malfunctions.
• User Directory – View contact information for system users.
• Leaderboard – Track your performance points relative to other engineers.

Available Sensors:
- Temperature (°C): Measures ambient temperature (indoor & outdoor)
- Humidity (%): Measures relative humidity (indoor & outdoor)
- Pressure (hPa): Measures atmospheric pressure (indoor)
- Dlight (Lux): Measures light intensity (outdoor)
- Distance (mm): Ultrasonic sensor for movement detection (indoor)
All sensors stream data in real time via MQTT protocol.

How to Earn Points:
- Complete fault simulator challenges.
- More complex faults award higher scores.
- Time efficiency matters: faster repairs = better ranking.
- You must complete all simulator steps for points; no partial credit.
- Your total score determines your leaderboard rank.

FAQ:
Q: Can I skip steps in the Fault Simulator and still get points?
A: No. You must complete all required steps before submitting. Partial credit is not available.

Q: Can I view or repeat previous faults?
A: Not at this stage. Handled faults cannot be revisited.

Q: Are the sensors live or simulated?
A: Both modes are supported. Simulation streams realistic data; live mode uses real sensors.

Q: How often is the leaderboard updated?
A: The leaderboard updates after each completed challenge.

About the CIM & Robotics Laboratory:
The CIM & Robotics Laboratory at Braude College of Engineering, established in 1997, offers hands-on experience in industrial robotics, automation, CNC machining, CAD/CAM tools, vision-based quality control, and PLCs. It bridges theory with real-world manufacturing skills.

About the Application:
OptiLine Dashboard is accessible from any device with secure login, supporting both live and simulated data. It provides engineers with real-time monitoring, analysis, troubleshooting, and a gamified performance experience.
"""


In [ ]:
"""
Live Data Helper Functions
--------------------------
Provides access to real-time sensor readings and leaderboard data.
All helper functions are documented and return data in a format suitable for inclusion in a Gemini prompt.
"""

def get_sensor_value(sensor_type: str, location: str, mode: str = "simulation"):
    """
    Retrieves the latest value for a specified sensor type and location.

    Args:
        sensor_type (str): Type of sensor ('temperature', 'humidity', 'pressure', 'dlight', 'distance')
        location (str): 'indoor' or 'outdoor'
        mode (str): 'simulation' or 'mqtt' (default is 'simulation')

    Returns:
        dict: {
            'value': float or None,
            'units': str,
            'timestamp': str (current time, ISO format),
            'error': str (if sensor or location not found)
        }
    """
    # Sensor availability check
    if sensor_type not in SENSORS_META:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Sensor '{sensor_type}' not found."}
    if location not in SENSORS_META[sensor_type]['locations']:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Sensor '{sensor_type}' is not available in {location}."}

    # Data source selection
    sensor_data = None
    try:
        if location == "indoor":
            data = next(indoor.get_live_data_stream(mode=mode))
        elif location == "outdoor":
            data = next(outdoor.get_live_data_stream(mode=mode))
        else:
            return {'value': None, 'units': '', 'timestamp': '', 'error': "Invalid location."}
        # Try matching by label/case-insensitive
        for key in data.keys():
            if key.lower() == sensor_type.lower():
                sensor_data = data[key]
                break
    except Exception as e:
        return {'value': None, 'units': '', 'timestamp': '', 'error': f"Error retrieving sensor data: {e}"}

    # Return structure
    if sensor_data is not None:
        return {
            'value': sensor_data,
            'units': SENSORS_META[sensor_type]['units'],
            'timestamp': datetime.now().isoformat(timespec="seconds"),
            'error': ""
        }
    else:
        return {
            'value': None,
            'units': SENSORS_META[sensor_type]['units'],
            'timestamp': '',
            'error': f"No data found for sensor '{sensor_type}' at '{location}'."
        }

def get_leaderboard_snapshot(username: str = ""):
    """
    Retrieves the leaderboard: top 5 users and the rank/score of a given user (if specified).

    Args:
        username (str): Username to highlight in the leaderboard (optional).

    Returns:
        dict: {
            'top5': list of (rank, username, score),
            'user_rank': (rank, score) or None
        }
    """
    try:
        top5, user_info = uc.get_leaderboard(username)
        return {
            "top5": top5,
            "user_rank": user_info  # (rank, score) tuple or None
        }
    except Exception as e:
        return {
            "top5": [],
            "user_rank": None,
            "error": f"Leaderboard error: {e}"
        }


In [ ]:
"""
Smart Intent Detection & Data Injection
---------------------------------------
This cell improves question handling for sensor values, normal ranges, and sensor descriptions.
It detects various user phrasings, including typos and synonyms, and injects the correct data into the Gemini prompt.
"""



def detect_sensor_intent(message: str):
    """
    Detects if the message requests a live sensor value, the normal range, or a description.
    Returns a dict with intent type and extracted info.
    """
    msg = message.lower()
    # Simple typo handling: 'temprature', 'humditty', etc.
    typo_map = {
        "temprature": "temperature",
        "temperture": "temperature",
        "temp": "temperature",
        "humditty": "humidity",
        "humdity": "humidity",
        "presure": "pressure",
        "d-light": "dlight",
        "distanc": "distance"
    }
    # Replace common typos in the message
    for typo, correct in typo_map.items():
        msg = msg.replace(typo, correct)

    # Sensor & location detection
    sensor_types = list(SENSORS_META.keys())
    locations = ["indoor", "outdoor"]
    found_sensor, found_location = None, None
    for s in sensor_types:
        if s in msg:
            found_sensor = s
            break
    for loc in locations:
        if loc in msg:
            found_location = loc
            break
    # Default location: prefer outdoor if available, else indoor
    if found_sensor and not found_location:
        if "outdoor" in SENSORS_META[found_sensor]["locations"]:
            found_location = "outdoor"
        else:
            found_location = "indoor"

    # --- Intent detection ---
    # 1. Live value (What is the X, show me the X, current X, etc.)
    if found_sensor and (
        re.search(r"(what is|show|current|get|value|reading|display)\s.*" + found_sensor, msg)
        or msg.strip() == found_sensor
    ):
        return {"intent": "live_value", "sensor": found_sensor, "location": found_location}
    # 2. Normal range (range, normal, valid, acceptable, safe, expected)
    if found_sensor and re.search(r"(range|normal|valid|acceptable|safe|expected)", msg):
        return {"intent": "normal_range", "sensor": found_sensor}
    # 3. Sensor description/explanation (what does X do, explain, why, purpose)
    if found_sensor and re.search(r"(what does|explain|purpose|why|used for|function)", msg):
        return {"intent": "sensor_description", "sensor": found_sensor}
    # Not detected
    return {"intent": None}

def enrich_prompt_with_sensor_data(intent_info, sensor_mode="simulation"):
    """
    Given detected intent, returns a string to inject to the Gemini prompt (live value, range, or description).
    """
    if not intent_info or not intent_info.get("sensor"):
        return ""
    sensor = intent_info["sensor"]
    if intent_info["intent"] == "live_value" and intent_info.get("location"):
        data = get_sensor_value(sensor, intent_info["location"], mode=sensor_mode)
        if data["error"]:
            return data["error"]
        return (
            f"Current value of {intent_info['location']} {sensor}: "
            f"{data['value']} {data['units']} (as of {data['timestamp']})."
        )
    elif intent_info["intent"] == "normal_range":
        rng = SENSORS_META[sensor]["normal_range"]
        units = SENSORS_META[sensor]["units"]
        return f"The normal range for {sensor} is {rng[0]} to {rng[1]} {units}."
    elif intent_info["intent"] == "sensor_description":
        desc = SENSORS_META[sensor]["description"]
        details = SENSORS_META[sensor]["details"]
        return f"{desc} {details}"
    return ""



In [ ]:
"""
Extended Intent Detection & Data Injection
-----------------------------------------
Detects user questions about the leaderboard (score, rank), active faults, and FAQ.
Injects the relevant data into the Gemini prompt.
"""



def detect_leaderboard_intent(message: str):
    """
    Detects if the message requests leaderboard info, score, or rank for a user.
    Returns dict with intent type and username if found.
    """
    msg = message.lower()
    # Common leaderboard/score words
    keywords = ["leaderboard", "score", "rank", "top", "points", "standing", "place", "position"]
    if any(k in msg for k in keywords):
        # Try to extract username (simple: after 'of', 'for', '@', etc.)
        username = ""
        user_search = re.search(r"(user ?name|of|for|@)\s*(\w+)", msg)
        if user_search:
            username = user_search.group(2)
        return {"intent": "leaderboard", "username": username}
    return {"intent": None}

def enrich_prompt_with_leaderboard(intent_info):
    """
    Enriches the Gemini prompt with leaderboard data:
    - Includes the current top 5 users (username + score).
    - If a specific username is provided (e.g. in "What is the score/rank of X?"),
      includes their rank and score even if they are not in the top 5.

    Args:
        intent_info (dict): Must contain 'intent' == "leaderboard" and optionally 'username'.

    Returns:
        str: English summary of leaderboard state for the prompt.
    """
    if not intent_info or intent_info["intent"] != "leaderboard":
        return ""
    username = intent_info.get("username", "")
    board = get_leaderboard_snapshot(username)
    if "error" in board:
        return board["error"]

    # Build top 5 table
    table = ", ".join([f"{rank}. {name} ({score})" for rank, name, score in board["top5"]])

    user_rank = ""
    ur = board.get("user_rank")
    # If user_rank is tuple (rank, score), show details if not in top 5
    if ur and isinstance(ur, (tuple, list)) and len(ur) == 2:
        rank, score = ur
        in_top5 = any(name == username for _, name, _ in board["top5"])
        if not in_top5:
            user_rank = f"User '{username}' is ranked {rank} with {score} points."
    elif username:
        user_rank = f"User '{username}' not found in the leaderboard."
    return f"Leaderboard (top 5): {table}. {user_rank}"



def detect_faults_intent(message: str):
    """
    Detects if the message requests info about faults/errors in the system.
    Returns dict with intent type.
    """
    msg = message.lower()
    if any(k in msg for k in ["fault", "error", "malfunction", "problem"]):
        return {"intent": "faults"}
    return {"intent": None}

def enrich_prompt_with_faults(intent_info):
    """
    Injects faults info into prompt.
    """
    if not intent_info or intent_info["intent"] != "faults":
        return ""
    try:
        faults = fb.get_active_faults()
        num_faults = len(faults) if faults else 0
        return (
            f"There are currently {num_faults} active faults in the system."
            if num_faults else "No active faults found at the moment."
        )
    except Exception as e:
        return f"Could not retrieve fault data: {e}"

def detect_faq_intent(message: str):
    """
    Detects if the message matches a known FAQ/guide question (simplified).
    Returns dict with intent type and matched question.
    """
    msg = message.lower()
    faq_qas = [
        ("can i skip steps", "No, you must complete all required steps before submitting. Partial credit is not available."),
        ("how do i earn points", "You earn points by completing Fault Simulator challenges. More difficult faults and faster completion earn more points."),
        ("are the sensors live", "The system supports both live and simulated sensor data."),
        ("how often does the leaderboard update", "The leaderboard updates after each completed challenge."),
        ("what is the fault simulator", "The Fault Simulator lets you practice troubleshooting simulated malfunctions for points."),
        ("about the cim & robotics lab", "The CIM & Robotics Lab at Braude College provides hands-on experience in industrial robotics and automation.")
    ]
    for q, a in faq_qas:
        if q in msg:
            return {"intent": "faq", "answer": a}
    return {"intent": None}

def enrich_prompt_with_faq(intent_info):
    """
    Injects FAQ answer into prompt.
    """
    if intent_info and intent_info["intent"] == "faq":
        return intent_info["answer"]
    return ""


In [ ]:
"""
Improved Main Entry: ask_optibot (with smart intent detection)
-------------------------------------------------------------
Handles user questions using all smart intent/enrichment functions before calling Gemini.
Injects live values, normal ranges, leaderboard data, faults, and FAQ answers into the prompt as needed.
"""

def ask_optibot(message: str, chat_history: list, sensor_mode: str = "simulation") -> tuple:
    """
    Main entry point for OptiBot Q&A (improved version).

    Args:
        message (str): User's latest message (question), in English.
        chat_history (list): List of previous messages: {"role": "user"/"assistant", "content": str}
        sensor_mode (str): 'simulation' or 'mqtt' (default: simulation)

    Returns:
        tuple:
            - Empty string (to clear Gradio input)
            - Updated chat_history (with the new user message and the bot response)
    """
    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    enrichment_notes = []

    # 1. Sensor intent
    sensor_intent = detect_sensor_intent(message)
    enrichment_notes.append(enrich_prompt_with_sensor_data(sensor_intent, sensor_mode))

    # 2. Leaderboard intent
    leaderboard_intent = detect_leaderboard_intent(message)
    enrichment_notes.append(enrich_prompt_with_leaderboard(leaderboard_intent))

    # 3. Faults intent
    faults_intent = detect_faults_intent(message)
    enrichment_notes.append(enrich_prompt_with_faults(faults_intent))

    # 4. FAQ intent
    faq_intent = detect_faq_intent(message)
    enrichment_notes.append(enrich_prompt_with_faq(faq_intent))

    # Filter out empty notes
    enrichment_notes = [n for n in enrichment_notes if n]

    # ---- Build prompt with highlighted enrichment ----
    prompt = OPTI_BOT_CONTEXT.strip() + "\n\n"
    prompt += (
        "When answering, ALWAYS use the data in the section 'Relevant live system data' below if it exists. "
        "If the answer is not available below, use your general world knowledge to answer. "
        "When a user's rank is provided, mention both the rank and score in your answer.\n"
    )
    if enrichment_notes:
        prompt += "\n===== Relevant live system data =====\n"
        prompt += "\n".join(enrichment_notes)
        prompt += "\n====================================\n"
    prompt += f"\nUser question: {message}\n"

    # Call Gemini API
    try:
        if not GEMINI_API_KEY:
            bot_response = "Error: Gemini API key is missing. Please contact your administrator."
        else:
            genai.configure(api_key=GEMINI_API_KEY)
            model = genai.GenerativeModel("gemini-1.5-flash")
            response = model.generate_content(prompt)
            bot_response = response.text.strip() if hasattr(response, "text") else str(response)
    except Exception as ex:
        bot_response = (
            "Sorry, I could not process your question due to a system error: "
            f"{ex}"
        )

    chat_history.append({"role": "assistant", "content": bot_response})
    return "", chat_history
